In [ ]:
import pyaudio
import wave
import numpy as np
import time
import librosa
from keras.models import load_model

# Load your emotion classification model
model = load_model('D:/Final Project/Speech/best_model.h5')

# Define audio recording parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 10
WAVE_OUTPUT_FILENAME = "D:/Final Project/Speech/output.wav"

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Open audio stream
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

print("Recording started...")

while True:
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording stopped.")
    
    # Save audio to a WAV file
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

    # Load and preprocess the recorded audio for emotion classification
    y, sr = librosa.load(WAVE_OUTPUT_FILENAME)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    test_auddf = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
    test_auddf = np.array([test_auddf])
    test_auddf = test_auddf[:, :, np.newaxis]

    # Predict emotions
    predictions = model.predict(test_auddf, batch_size=1)
    predicted_class = np.argmax(predictions)
    
    # Get the corresponding emotion label
    emotions = ['angry', 'calm', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    predicted_emotion = emotions[predicted_class]
    
    print(f"Predicted emotion: {predicted_emotion}")

    # Wait for 10 seconds before recording again
    time.sleep(10)

# Stop and close the audio stream
stream.stop_stream()
stream.close()
audio.terminate()
